# Experiment

## Install Python dependencies

In [ ]:
!pip install onnx onnxruntime tf2onnx lakefs==0.7.1 s3fs==2024.10.0

Import the dependencies for the model training code:

In [ ]:
import numpy as np
import pandas as pd
import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
import tf2onnx
import onnx
import pickle
from pathlib import Path
import lakefs
import os
import s3fs

The output might show TensorFlow messages, such as a "Could not find TensorRT" warning. You can ignore these messages.


## Define lakeFS Storage and Repository information

In [ ]:
lakefs_storage_options={
      "key": os.environ.get('LAKECTL_CREDENTIALS_ACCESS_KEY_ID'),
      "secret": os.environ.get('LAKECTL_CREDENTIALS_SECRET_ACCESS_KEY'),
      "client_kwargs": {
         "endpoint_url": os.environ.get('LAKECTL_SERVER_ENDPOINT_URL')
      }
}

repo_name = os.environ.get('LAKEFS_REPO_NAME')
mainBranch = "main"
trainingBranch = "train01"

repo = lakefs.Repository(repo_name)
print(repo)

## Create Training branch in lakeFS and load the CSV data to the training branch in lakeFS

In [ ]:
branchTraining = repo.branch(trainingBranch).create(source_reference=mainBranch, exist_ok=True)

obj = branchTraining.object(path='data/train.csv')
with open('data/train.csv', mode='rb') as reader, obj.writer(mode='wb', metadata={'using': 'python_wrapper', 'source':'Fraud Detection Demo'}) as writer:
    writer.write(reader.read())

obj = branchTraining.object(path='data/validate.csv')
with open('data/validate.csv', mode='rb') as reader, obj.writer(mode='wb', metadata={'using': 'python_wrapper', 'source':'Fraud Detection Demo'}) as writer:
    writer.write(reader.read())

obj = branchTraining.object(path='data/test.csv')
with open('data/test.csv', mode='rb') as reader, obj.writer(mode='wb', metadata={'using': 'python_wrapper', 'source':'Fraud Detection Demo'}) as writer:
    writer.write(reader.read())

## Load the CSV data from the training branch in lakeFS

The CSV data that you use to train the model contains the following fields:

* **distancefromhome** - The distance from home where the transaction happened.
* **distancefromlast_transaction** - The distance from the last transaction that happened.
* **ratiotomedianpurchaseprice** - The ratio of purchased price compared to median purchase price.
* **repeat_retailer** - If it's from a retailer that already has been purchased from before.
* **used_chip** - If the credit card chip was used.
* **usedpinnumber** - If the PIN number was used.
* **online_order** - If it was an online order.
* **fraud** - If the transaction is fraudulent.

In [ ]:
# Set the input (X) and output (Y) data. 
# The only output data is whether it's fraudulent. All other fields are inputs to the model.

feature_indexes = [
    1,  # distance_from_last_transaction
    2,  # ratio_to_median_purchase_price
    4,  # used_chip
    5,  # used_pin_number
    6,  # online_order
]

label_indexes = [
    7  # fraud
]

df = pd.read_csv(f"s3://{repo_name}/{trainingBranch}/data/train.csv", storage_options=lakefs_storage_options)
X_train = df.iloc[:, feature_indexes].values
y_train = df.iloc[:, label_indexes].values

df = pd.read_csv(f"s3://{repo_name}/{trainingBranch}/data/validate.csv", storage_options=lakefs_storage_options)
X_val = df.iloc[:, feature_indexes].values
y_val = df.iloc[:, label_indexes].values

df = pd.read_csv(f"s3://{repo_name}/{trainingBranch}/data/test.csv", storage_options=lakefs_storage_options)
X_test = df.iloc[:, feature_indexes].values
y_test = df.iloc[:, label_indexes].values


# Scale the data to remove mean and have unit variance. The data will be between -1 and 1, which makes it a lot easier for the model to learn than random (and potentially large) values.
# It is important to only fit the scaler to the training data, otherwise you are leaking information about the global distribution of variables (which is influenced by the test set) into the training set.

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

obj = branchTraining.object(path='artifact/test_data.pkl')
with obj.writer("wb") as handle:
    pickle.dump((X_test, y_test), handle)
obj = branchTraining.object(path='artifact/scaler.pkl')
with obj.writer("wb") as handle:
    pickle.dump(scaler, handle)

# Since the dataset is unbalanced (it has many more non-fraud transactions than fraudulent ones), set a class weight to weight the few fraudulent transactions higher than the many non-fraud transactions.
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.ravel())
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

## Build the model

The model is a simple, fully-connected, deep neural network, containing three hidden layers and one output layer.

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=len(feature_indexes)))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Train the model

Training a model is often the most time-consuming part of the machine learning process.  Large models can take multiple GPUs for days.  Expect the training on CPU for this very simple model to take a minute or more.

In [ ]:
# Train the model and get performance
import os
import time

start = time.time()
epochs = 2
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_val, y_val),
    verbose=True,
    class_weight=class_weights
)
end = time.time()
print(f"Training of model is complete. Took {end-start} seconds")

## Save the model file

In [ ]:
import tensorflow as tf

# Normally we use tf2.onnx.convert.from_keras.
# workaround for tf2onnx bug https://github.com/onnx/tensorflow-onnx/issues/2348

# Wrap the model in a `tf.function`
@tf.function(input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')])
def model_fn(x):
    return model(x)

# Convert the Keras model to ONNX
model_proto, _ = tf2onnx.convert.from_function(
    model_fn,
    input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name='dense_input')]
)

# Save the model as ONNX for easy use of ModelMesh
os.makedirs("models/fraud/1", exist_ok=True)
onnx.save(model_proto, "models/fraud/1/model.onnx")

The output might include TensorFlow messages related to GPUs. You can ignore these messages.

## Confirm the model file was created successfully

The output should include the model name, size, and date. 

In [ ]:
! ls -alRh ./models/

## Test the model

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import pickle
import onnxruntime as rt

Load the test data and scaler from the training branch in lakeFS:

In [ ]:
obj = branchTraining.object(path='artifact/scaler.pkl')
with obj.reader('rb') as handle:
    scaler = pickle.load(handle)
obj = branchTraining.object(path='artifact/test_data.pkl')
with obj.reader('rb') as handle:
    (X_test, y_test) = pickle.load(handle)

Create an ONNX inference runtime session and predict values for all test inputs:

In [ ]:
sess = rt.InferenceSession("models/fraud/1/model.onnx", providers=rt.get_available_providers())
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name
y_pred_temp = sess.run([output_name], {input_name: X_test.astype(np.float32)}) 
y_pred_temp = np.asarray(np.squeeze(y_pred_temp[0]))
threshold = 0.95
y_pred = np.where(y_pred_temp > threshold, 1, 0)

Show the results:

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np

y_test_arr = y_test.squeeze()
correct = np.equal(y_pred, y_test_arr).sum().item()
acc = (correct / len(y_pred)) * 100
precision = precision_score(y_test_arr, np.round(y_pred))
recall = recall_score(y_test_arr, np.round(y_pred))

print(f"Eval Metrics: \n Accuracy: {acc:>0.1f}%, "
      f"Precision: {precision:.4f}, Recall: {recall:.4f} \n")

c_matrix = confusion_matrix(y_test_arr, y_pred)
ConfusionMatrixDisplay(c_matrix).plot()

## Example: Is Sally's transaction likely to be fraudulent?

Here is the order of the fields from Sally's transaction details:
* distance_from_last_transaction
* ratio_to_median_price
* used_chip 
* used_pin_number
* online_order 

In [ ]:
sally_transaction_details = [
    [0.3111400080477545,
    1.9459399775518593,
    1.0,
    0.0,
    0.0]
    ]

prediction = sess.run([output_name], {input_name: scaler.transform(sally_transaction_details).astype(np.float32)})

print("Is Sally's transaction predicted to be fraudulent? (true = YES, false = NO) ")
print(np.squeeze(prediction) > threshold)

print("How likely was Sally's transaction to be fraudulent? ")
print("{:.5f}".format(100 * np.squeeze(prediction)) + "%")